In [1]:
import glob
import os
import pandas as pd

define constants

In [2]:
keys = ["top","tok","mess","rush","sil","s_m","s_n","er","EOS",]
column_names = ["Mwfsending", "Mwfreceiving", "Mwfreturning", "Mcirctime", "top","tok","mess","rush","sil","s_m","s_n","er"]
path_to_write_data = os.getcwd()+"\\"
path_to_data = path_to_write_data+"data\\"
filename = "results.txt"

define functions

In [3]:
def createColumnNames():
    result = ""
    for column_name in column_names:
        result = result + column_name + " "
    return result

In [4]:
def processFilename(filename):
    result = ""
    for x in range(0, len(keys)-1):
        result = result + filename[filename.find(keys[x])+len(keys[x]): filename.find(keys[x+1])]+" "
    return result

In [5]:
def appendStr(appendix):
    if os.path.exists(path_to_write_data+filename):
        append_write = 'a' # append if already exists
    else:
        append_write = 'w' # make a new file if not

    file_with_results = open(path_to_write_data+filename, append_write)
    if append_write == "w":
            file_with_results.write(createColumnNames()+"\n")
    file_with_results.write(appendix+"\n")
    file_with_results.close()

In [6]:
def collectResults(filename):
    names = ['Created', 'Sent', 'Received', 'Returned']
    data = pd.read_table(filename, sep=' ', names=names)
    data = data.drop(data.index[0])
    data = data.apply(pd.to_numeric, errors='coerce')
    wait_for_sending = data["Sent"] - data["Created"]
    wait_for_receiving = data["Received"] - data["Created"]
    circulated_before_receiving = data["Received"] - data["Sent"]
    time = data["Created"] - min(data["Created"])
    wait_for_returning = data["Returned"] - data["Created"]
    circulated_before_return = data["Returned"] - data["Sent"]
    not_returned = data.loc[data["Returned"]==0].index
    circulated_before_return_good = circulated_before_return.drop(not_returned)
    wait_for_returning_good = wait_for_returning.drop(not_returned)
    time_good = time.drop(not_returned)
    result = str(wait_for_sending.median())
    result += " "
    result += str(wait_for_receiving.median())
    result += " "
    result += str(wait_for_returning_good.median())
    result += " "
    result += str(circulated_before_return_good.median())
    result += " "
    return result

In [7]:
def takeDataFromFile(elem):
    name = elem.replace(path_to_data, "")
    result = collectResults(elem)+processFilename(name)
    print result
    os.rename(elem, path_to_data+"used\\"+name)
    return result

create dataset with all measurements

In [8]:
t = glob.glob(path_to_data+"*.txt")
print t

['D:\\Git\\TokenRing\\research\\data\\103222410top7tok1mess1000rushtruesiltrues_m10s_n10erfalseEOS.txt', 'D:\\Git\\TokenRing\\research\\data\\103222411top7tok2mess1000rushtruesiltrues_m10s_n10erfalseEOS.txt', 'D:\\Git\\TokenRing\\research\\data\\103222411top7tok3mess1000rushtruesiltrues_m10s_n10erfalseEOS.txt', 'D:\\Git\\TokenRing\\research\\data\\103222411top7tok4mess1000rushtruesiltrues_m10s_n10erfalseEOS.txt', 'D:\\Git\\TokenRing\\research\\data\\103222412top7tok5mess1000rushtruesiltrues_m10s_n10erfalseEOS.txt', 'D:\\Git\\TokenRing\\research\\data\\103222412top7tok6mess1000rushtruesiltrues_m10s_n10erfalseEOS.txt', 'D:\\Git\\TokenRing\\research\\data\\103222412top7tok7mess1000rushtruesiltrues_m10s_n10erfalseEOS.txt', 'D:\\Git\\TokenRing\\research\\data\\103222413top8tok1mess1000rushtruesiltrues_m10s_n10erfalseEOS.txt', 'D:\\Git\\TokenRing\\research\\data\\103222414top8tok2mess1000rushtruesiltrues_m10s_n10erfalseEOS.txt', 'D:\\Git\\TokenRing\\research\\data\\103222414top8tok3mess1000r

In [9]:
for elem in t:
    appendStr(takeDataFromFile(elem))

124800927.0 125303824.0 125651212.0 763940.0 7 1 1000 true true 10 10 false 
63349476.0 63723861.0 64104492.0 769295.0 7 2 1000 true true 10 10 false 
43144774.5 43582300.0 44002422.0 778666.0 7 3 1000 true true 10 10 false 
35802782.0 36256148.0 36769086.5 760370.5 7 4 1000 true true 10 10 false 
24678125.5 25104049.0 25426671.0 692767.0 7 5 1000 true true 10 10 false 
26851250.0 27238128.0 27840534.5 849838.5 7 6 1000 true true 10 10 false 
17609001.0 17975576.5 18322964.0 704146.0 7 7 1000 true true 10 10 false 
165679538.5 166090514.0 166602112.5 857871.0 8 1 1000 true true 10 10 false 
85231990.5 85811863.0 86368085.0 835336.0 8 2 1000 true true 10 10 false 
69381123.0 69886698.5 70493121.0 983037.0 8 3 1000 true true 10 10 false 
44868995.0 45515577.0 45999733.0 862334.0 8 4 1000 true true 10 10 false 
50849995.0 51463111.5 52163464.0 1381964.0 8 5 1000 true true 10 10 false 
44196084.5 44962701.5 45563769.5 1449344.0 8 6 1000 true true 10 10 false 
24444303.0 24876251.0 25331401

In [10]:
print "Finished processing!"

Finished processing!
